In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import defaultdict

#detect for gpu
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU detected and configured.")
else:
    print("No GPU detected.")

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/deep learning/breast cancer/cancer dataset'
img_size = (224, 224)
limit_per_class = 5000

#Load Img
data = []
labels = []
class_counts = defaultdict(int)

for class_name in sorted(os.listdir(dataset_dir)):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        images = sorted(os.listdir(class_path))[:limit_per_class]
        for img_name in images:
            img_path = os.path.join(class_path, img_name)
            try:
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0
                data.append(img_array)
                labels.append(class_name)
                class_counts[class_name] += 1
            except Exception as e:
                print(f"Skipping {img_path}: {e}")

print(f"Images loaded per class: {dict(class_counts)}")

#Convert to arrays
data = np.array(data, dtype='float32')
labels = np.array(labels)

#Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_classes = len(le.classes_)
labels = to_categorical(labels_encoded, num_classes=num_classes)

#split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

#Load DenseNet121 base
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

#model with regularization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.7),
    Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001))
])

#Compile
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#Train
print("Training model...")
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

#Evaluate
score = model.evaluate(X_test, y_test, verbose=0)
print(f'\nTest Accuracy: {score[1]*100:.2f}%')
print(f'Final Training Accuracy: {history.history["accuracy"][-1]*100:.2f}%')
print(f'Final Validation Accuracy: {history.history["val_accuracy"][-1]*100:.2f}%')


✅ GPU detected and configured.
✅ Images loaded per class: {'0_NoCancer': 1180, '1_Cancer': 1178}
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
🔁 Training model...
Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.6437 - loss: 1.6549 - val_accuracy: 0.8164 - val_loss: 1.1425
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 243ms/step - accuracy: 0.7958 - loss: 1.1410 - val_accuracy: 0.8305 - val_loss: 0.9453
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.8222 - loss: 0.9310 - val_accuracy: 0.8319 - val_loss: 0.8344
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 212ms/step - accuracy: 0.8222 - loss: 0.8377 - val_accuracy: 0.8305 - val_loss: 0.7824
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 210ms/step - accuracy: 0.8320 - loss: 0.7705 - val_accuracy: 0.8362 - val_loss: 0.7345
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 206ms/step - accuracy: 0.8257 - loss: 0.7410 - val_accuracy: 0.8291 - val_loss: 0.7223
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy